In [53]:
#Data munging
#using Dasgupta script to create risk factor ratios in original dataset
import numpy as np
import pandas as pd
import os
%matplotlib inline
os.chdir('P:\Framingham hip\Framingham Offsping Cohort datasets of Interest')
os.getcwd()

'P:\\Framingham hip\\Framingham Offsping Cohort datasets of Interest'

In [54]:
offspring = pd.read_csv('offspring_full_2000_redone_columns_bmi.csv')

In [55]:
offspring['idtype'].value_counts()

1    5013
7     494
Name: idtype, dtype: int64

In [56]:
pd.crosstab(offspring.idtype, offspring.examyr4)

examyr4,,1987,1988,1989,1990,1991,2011,2012,2013,2014
idtype,,,,,,,,,,
1,1087,503,1214,1069,1093,47,0,0,0,0
7,203,0,0,0,0,0,45,172,69,5


In [57]:
offspring =offspring[offspring.idtype != 7]
pd.crosstab(offspring.idtype, offspring.examyr4)

examyr4,,1987,1988,1989,1990,1991
idtype,,,,,,
1,1087,503,1214,1069,1093,47


In [58]:
#rename columns to homogenize with 'original' dataset
offspring.rename(columns={'Period_STOP_AGE':'age_periods_stop'},inplace=True)
offspring.rename(columns={'Estrogen_conj_1990':'estrogen_1990'},inplace=True)

#rename diabetes coulumns to proper format.
offspring.rename(columns={'DIAB2':'diab_1980'},inplace=True)
offspring.rename(columns={'DIAB4':'diab_1990'},inplace=True)
offspring.rename(columns={'DIAB7':'diab_2000'},inplace=True)
#Beta_1980 beta blockers in 1980 has a funny coding system.  3 = NO and 4 = YES
recode ={3:0,4:1,5:0}


In [59]:
offspring['Beta_1980'].value_counts()

3.0    3595
4.0     152
5.0      28
Name: Beta_1980, dtype: int64

In [60]:
offspring['Beta_1980'].replace(recode, inplace=True)

In [61]:
offspring['Beta_1980'].value_counts()

0.0    3623
1.0     152
Name: Beta_1980, dtype: int64

In [62]:
recode_estrogen = {8:np.NaN, 2:0}  #estrogen = 8 indicates patient is a man.  Analysis makes most sense to recode as NaN.
# estrogen = 2 means they took it in the past but not now.  Recode as 0.

In [63]:
offspring['Estrogen_1980'].replace(recode_estrogen, inplace=True)
offspring['estrogen_1990'].replace(recode_estrogen, inplace=True)
offspring['Estrogen_2000'].replace(recode_estrogen, inplace=True)

In [64]:
offspring['Steroid_2000'].value_counts()   #Steroid is sparsely populated and has no zeros.  I think we will ignore this.
# if necessary, can recode steroid as 0 if person attended that visit.
#only available 1 year

1.0    55
Name: Steroid_2000, dtype: int64

In [65]:
## Fix type of age columns
age_cols = [u for u in offspring.columns if u.find('age')>-1]
for col in age_cols:
    offspring[col] = pd.to_numeric(offspring[col], errors='coerce')

blah = ((offspring.age2 > offspring.age_periods_stop) & (offspring.age_periods_stop < 45)).astype(int)
blah[pd.isnull(offspring.age2) | pd.isnull(offspring.age_periods_stop)] = np.nan
offspring['menopause_1980'] = blah.copy()

blah = ((offspring.age4 > offspring.age_periods_stop) & (offspring.age_periods_stop < 45)).astype(int)
blah[pd.isnull(offspring.age4) | pd.isnull(offspring.age_periods_stop)] = np.nan
offspring['menopause_1990'] = blah.copy()

blah = ((offspring.age7 > offspring.age_periods_stop) & (offspring.age_periods_stop < 45)).astype(int)
blah[pd.isnull(offspring.age7) | pd.isnull(offspring.age_periods_stop)] = np.nan
offspring['menopause_2000'] = blah.copy()

In [66]:
# Create tidy data set
offspring_long = offspring.melt(id_vars = ['PID'])

risk_factors = offspring_long.loc[offspring_long['variable'].str.contains('[0-9]{4}$')]
risk_factors['variable'] = risk_factors['variable'].str.replace('_wine','wine') # rationalize wine
bl = risk_factors['variable'].str.split('_', expand=True).iloc[:,:2] # Split variables and year
bl = bl.rename(columns = {0:'vars', 1:'year'})
risk_factors = risk_factors.join(bl)
risk_factors['value'] = pd.to_numeric(risk_factors['value'], errors='coerdce') # Make numeric
risk_factors['vars'] = risk_factors['vars'].str.lower() # Make lower case
risk_factors = risk_factors.drop('variable',1)


bl = risk_factors.pivot_table(index = 'PID', values = 'value',
                              columns = ['year','vars'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [67]:
# Aggregate wines for 2000
d = bl['2000'][['redwine','whitewine']]  #no otherwine in offspring dataset
d['wine'] = d.sum(axis = 1)

bl['2000','wine'] = d['wine']
bl = bl.sort_index(axis=1)
bl = bl.drop([('2000','redwine'),('2000','whitewine')],1)

In [68]:
bl

year     1980                                                               \
vars     beer beta        bmi cocktail diab estrogen menopause smoke  wine   
PID                                                                          
2924      0.0  0.0  22.648361      0.0  0.0      0.0       0.0   0.0   1.0   
3297     36.0  0.0  26.476584      0.0  0.0      NaN       NaN   1.0   0.0   
4061     18.0  1.0  31.644676      0.0  0.0      NaN       NaN   1.0   0.0   
5350      NaN  NaN        NaN      NaN  NaN      NaN       NaN   NaN   NaN   
7248      NaN  NaN        NaN      NaN  NaN      NaN       NaN   NaN   NaN   
8059      0.0  0.0  23.840380      0.0  0.0      0.0       0.0   2.0   0.0   
9458      0.0  0.0  18.033647      2.0  0.0      0.0       0.0   0.0   1.0   
14463     0.0  0.0  26.129085      7.0  0.0      0.0       1.0   3.0   0.0   
14764     NaN  NaN        NaN      NaN  NaN      NaN       NaN   NaN   NaN   
15059     0.0  0.0  16.839087      0.0  0.0      0.0       1.0   0.0   1.0   
18844     1.0  0.0  21.840750      0.0  0.0      0.0       1.0   1.0   1.0   
20206     1.0  0.0  24.598274      4.0  0.0      0.0       0.0   3.0   0.0   
22267     0.0  0.0  30.717181      0.0  0.0      NaN       NaN   3.0   0.0   
22996     NaN  NaN        NaN      NaN  NaN      NaN       NaN   NaN   NaN   
27195     0.0  0.0  24.705440      2.0  0.0      0.0       0.0   0.0   2.0   
27363     0.0  0.0  19.975780      0.0  0.0      0.0       0.0   3.0   1.0   
32688    10.0  0.0  26.103862      1.0  0.0      NaN       NaN   3.0   5.0   
33693     1.0  0.0  26.900880      0.0  0.0      NaN       NaN   3.0   0.0   
35867     0.0  0.0  20.656033      0.0  0.0      0.0       0.0   0.0   0.0   
38579     0.0  0.0  26.246809      0.0  0.0      NaN       NaN   0.0   2.0   
43540     0.0  0.0  19.415361      0.0  0.0      0.0       1.0   1.0   0.0   
44583     NaN  NaN        NaN      NaN  NaN      NaN       NaN   NaN   NaN   
46959     3.0  0.0  23.928540      1.0  0.0      NaN       NaN   3.0   0.0   
47291     0.0  0.0  20.604900      0.0  0.0      0.0       0.0   1.0   0.0   
48800     NaN  NaN        NaN      NaN  NaN      NaN       NaN   NaN   NaN   
49050     6.0  0.0  29.289609     28.0  0.0      NaN       NaN   3.0   0.0   
50122     0.0  0.0  21.124174      0.0  0.0      NaN       NaN   1.0  28.0   
51504     NaN  NaN        NaN      NaN  NaN      NaN       NaN   NaN   NaN   
52532     0.0  0.0  21.791427      0.0  0.0      0.0       0.0   1.0   0.0   
55025     0.0  0.0  20.806948      1.0  0.0      0.0       0.0   1.0   1.0   
...       ...  ...        ...      ...  ...      ...       ...   ...   ...   
9939621  12.0  0.0  33.206243      3.0  0.0      NaN       NaN   3.0   1.0   
9939888  24.0  0.0  24.759029     32.0  0.0      NaN       NaN   3.0   0.0   
9940525   NaN  NaN        NaN      NaN  NaN      NaN       NaN   NaN   NaN   
9940870   0.0  0.0  25.220102      7.0  0.0      NaN       NaN   0.0   7.0   
9941870   0.0  0.0  20.363903      0.0  0.0      0.0       0.0   3.0  28.0   
9943257   NaN  NaN        NaN      NaN  NaN      NaN       NaN   NaN   NaN   
9945338   1.0  0.0  22.988938      1.0  0.0      0.0       0.0   3.0   0.0   
9947765   0.0  0.0  22.119098      0.0  0.0      0.0       0.0   0.0   2.0   
9947948   0.0  0.0  22.137495      0.0  0.0      0.0       0.0   3.0   0.0   
9948725   0.0  0.0  37.379107      3.0  0.0      0.0       0.0   0.0  14.0   
9951579   0.0  0.0  23.007859      0.0  0.0      NaN       NaN   1.0   0.0   
9957447   NaN  NaN        NaN      NaN  NaN      NaN       NaN   NaN   NaN   
9958150   0.0  0.0  21.682869      0.0  0.0      0.0       0.0   0.0   0.0   
9965528   1.0  0.0  23.682465      2.0  0.0      NaN       NaN   0.0   1.0   
9967264   NaN  NaN        NaN      NaN  NaN      NaN       NaN   NaN   NaN   
9973742   NaN  NaN        NaN      NaN  NaN      NaN       NaN   NaN   NaN   
9975906   0.0  0.0  21.908598      0.0  0.0      0.0       0.0   3.0   0.0   
9977699  14.0  0.0  19

In [69]:
#==============================================================================
# Fix smoking in 1980
#==============================================================================

bl['1980','smoke'] = np.where(bl['1980','smoke']>0, 1, bl['1980','smoke'])


In [70]:
# Aggregate drinks to get total drinks, then create 
# RF_ETOH = 1 if drinks > 3
#==============================================================================
for u in ['1980','1990','2000']:
    d = bl[u]
    drinks = d[['beer','wine','cocktail']].sum(axis=1)
    drinks[drinks <= 3] = 0
    drinks[drinks > 3] = 1
    bl[u,'rf_etof']= drinks

In [71]:
list(bl.columns)


[('1980', 'beer'),
 ('1980', 'beta'),
 ('1980', 'bmi'),
 ('1980', 'cocktail'),
 ('1980', 'diab'),
 ('1980', 'estrogen'),
 ('1980', 'menopause'),
 ('1980', 'smoke'),
 ('1980', 'wine'),
 ('1990', 'beer'),
 ('1990', 'beta'),
 ('1990', 'bmi'),
 ('1990', 'cocktail'),
 ('1990', 'diab'),
 ('1990', 'estrogen'),
 ('1990', 'menopause'),
 ('1990', 'premarin'),
 ('1990', 'smoke'),
 ('1990', 'wine'),
 ('2000', 'beer'),
 ('2000', 'beta'),
 ('2000', 'bisphosphonates'),
 ('2000', 'bmi'),
 ('2000', 'cocktail'),
 ('2000', 'diab'),
 ('2000', 'estrogen'),
 ('2000', 'menopause'),
 ('2000', 'smoke'),
 ('2000', 'steroid'),
 ('2000', 'wine'),
 ('1980', 'rf_etof'),
 ('1990', 'rf_etof'),
 ('2000', 'rf_etof')]

In [72]:
# Summary
#==============================================================================

bl['PID'] = bl.index
risk_factors = bl.melt(id_vars = 'PID')
pd.pivot_table(risk_factors, values ='value', index = 'year', 
               columns = 'vars', aggfunc=np.nanmean).to_excel('RiskFactors_offspring.xlsx')

In [73]:
bl.to_csv('offspring_munging_done_10-9-17.csv')